# Uso de RAG para recuperar conhecimento organizacional

## Objetivo: Criar uma rag

In [ ]:
%%capture --no-stderr
%pip install langchain langchain-community langchain-text-splitters dotenv langchain-google-genai chromadb pypdf langchain-mistralai ipywidgets unstructured langchain-google-vertexai pdfminer.six sentence_transformers langchain-huggingface unstructured[local-inference]

## Imports

### Bibliotecas suporte

In [33]:
import os
import getpass
from dotenv import load_dotenv
from typing_extensions import TypedDict, List

### Parser e chunking

In [34]:
from langchain.schema import Document
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter

### LLM Models

In [35]:
#Groq
from langchain_groq import ChatGroq

#Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

### Embedding Models

In [36]:
#Gemini
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings
#MistralAI
from langchain_mistralai import MistralAIEmbeddings

### Vector Store

In [37]:
from langchain_community.vectorstores import Chroma

### Langchain

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.indexes import VectorstoreIndexCreator

## Codigo da RAG

### Configurar chaves das API's

In [39]:
load_dotenv()  


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")
_set_env("GOOGLE_API_KEY")

folder = "/home/gui/rag-knowledge-managment/docs/pdfs"

In [40]:
def carregar_documentos(pasta):
    docs = []
    for arquivo in os.listdir(pasta):
        if arquivo.lower().endswith('.pdf'):
            caminho = os.path.join(pasta, arquivo)
            docs.extend(UnstructuredPDFLoader(caminho).load())
    return docs

In [41]:
def get_embeddings():
    return MistralAIEmbeddings(model="mistral-embed")

In [42]:
def dividir_documentos(documentos, chunk_size=1000, chunk_overlap=200):
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documentos)

In [43]:
def criar_e_armazenar_embeddings(docs, collection_name):
    store = Chroma.from_documents(
        docs,
        embedding=get_embeddings(),
        collection_name=collection_name
    )
    return store

In [44]:
def configurar_llm(modelo):
    match modelo:
        case "gemini":
            return ChatGoogleGenerativeAI(model="gemini-2.5-pro-preview-03-25")
        case "groq":
            return ChatGroq(model="llama-3.3-70b-versatile")

In [45]:
def consultar_groq(vectorstore, query):
    # Recupera os chunks mais relevantes
    docs = vectorstore.similarity_search(query, k=3)
    contexto = "\n".join([doc.page_content for doc in docs])

    # Consulta LLM via Groq
    llm = configurar_llm("groq")
    prompt = f"Responda a pergunta usando apenas as informações abaixo:\n\n{contexto}\n\nPergunta: {query}"
    resposta = llm.invoke(prompt)
    return resposta

In [46]:
docs = carregar_documentos(folder)
chunks = dividir_documentos(docs, chunk_size=1000, chunk_overlap=200)
store = criar_e_armazenar_embeddings(chunks, collection_name="km-db")

# 2. Consultar usando Groq (substitua pela sua API Key real!)
pergunta = "Quais são os objetivos especificos do estudo de boas praticas de gestão do conhecimento em startups?"
resposta = consultar_groq(store, pergunta)
print("Resposta da LLM:", resposta)

Created a chunk of size 1202, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1577, which is longer than the specified 1000
Created a chunk of size 1068, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1035, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1309, which is longer than the specified 1000
Created a chunk of size 1514, which is longer than the specified 1000
Created a chunk of size 1713, which is longer than the specified 1000
Created a chunk of size 1597, which is longer than the specified 1000
Created a chunk of size 1154, which is longer than the specified 1000
Created a chunk of size 1958, which is longer than the specified 1000


Resposta da LLM: content='De acordo com o texto fornecido, os objetivos específicos do estudo de boas práticas de gestão do conhecimento em startups não são explicitamente listados. No entanto, podemos inferir que os objetivos incluem:\n\n1. **Identificar desafios**: Entender como a rotina acelerada e a alta rotatividade de funcionários impactam a gestão do conhecimento em startups.\n2. **Compreender a importância da gestão do conhecimento**: Reconhecer a gestão do conhecimento como um dos pilares fundamentais para o sucesso das startups, juntamente com a organização ágil e as capacidades dinâmicas.\n3. **Descobrir práticas eficazes**: Explorar iniciativas bem-sucedidas ou ideias que melhoram a gestão de conhecimento em startups, como a colaboração e o compartilhamento de conhecimento entre os times.\n4. **Entender a relação com a inovação**: Compreender como a gestão do conhecimento em startups está alinhada com os princípios da inovação aberta e como pode ser um diferencial competiti